# Textähnlichkeitssuche im Linkkontext
Um neue archivierungswürdige Webinhalte für eine Sammlung zu finden, werten wir die Links in archivierten Websites aus. Mit Hilfe des [Archives Unleashed Toolkit](https://archivesunleashed.org/aut/) haben wir neben dem Linkziel auch den Textkontext des Links aus den archivierten HTML-Dokumenten extrahiert. Zusätzlich haben wir die Links mit thematischen Schlagworten aus dem [OPAC der Bayerischen Staatsbibliothek](https://opacplus.bsb-muenchen.de) angereichert, mit denen Bibliothekar\*innen die archivierten Ausgangswebsites beschrieben haben.  
Anschließend haben wir mit [Sentence-Transformers](https://www.sbert.net) Satzembeddings für den Linkkontext erzeugt und mit der Suchmaschine [vespa](https://vespa.ai/) indexiert.  
Bibliothekar\*innen sollen so gezielt im Linkkontext nach Webinhalten zu bestimmten Themen suchen oder anhand des Schlagwortfilters die Links durchstöbern können, die von einer archivierten Website zu einem Thema ausgehen.  
Anders als bei der klassischen Volltextsuche können Nutzer\*innen hier mit natürlichsprachigen Sätzen statt einzelner Begriffe suchen. Das Sentence-Transformer Modell bildet die Anfrage und den Linkkontext so auf einen Vektorraum ab, dass Sätze mit ähnlichem semantischem Gehalt nahe beieinander liegen. Dadurch, dass die einzelnen Begriffe in ihrem Satzzusammenhang ausgewertet werden, kann das Modell auch kontextabhängig zwischen verschiedenen Bedeutungen eines Begriffs unterscheiden.

In [ ]:
from vespa.application import Vespa
from ipywidgets import HTML, Text, Button, Layout, SelectMultiple, Accordion
from sentence_transformers import SentenceTransformer
from util.vespa_helper.queries import get_facets, get_links
from util.vespa_helper.result_formatter import write_domain_table
import re

display(HTML('''<style>
    .widget-label { min-width: 25ex !important; }
</style>
'''))


vespa_url = "http://localhost:8080/search"
    
annotations = get_facets(vespa_url, "src_annotations")
   

def display_results(b):
    links = get_links(vespa_url, query_input.value, facets.value, 100)
    groups = links.hits[0].get("children", [])
    for group in groups:
        if group.get("label") == "domain":
            domain_groups = group.get("children", [])
            domain_tables = []
            for i, domain_group in enumerate(domain_groups):
                acc.set_title(i, f"{i+1}. {domain_group.get('value')}")
                domain_table = write_domain_table(domain_group.get("children", [])[0])
                domain_tables.append(HTML(domain_table))
            acc.children = domain_tables
              

facets = SelectMultiple(
    options=annotations,
    description='<b>Filter nach Schlagwort: </b>',
    disabled=False,
    layout=Layout(width="70%", height="100%")
)  
default_query = "Entdecken Sie die Vielfalt der bayerischen Sprache!"
query_input = Text(value=default_query, description="<b>Suche nach: </b>", layout=Layout(width="70%"))
acc = Accordion(selected_index=None)
button = Button(description="Suche starten")
button.on_click(display_results)
display(facets,query_input, button, acc)
